In [3]:
import numpy as np
from sklearn.model_selection import KFold

In [6]:
train = np.arange(5,15)
kf = KFold(5, shuffle = False)
# for a,b in kf.split(p_input, p_target):
#     print(a,'+',b)

fold = 0
scores = np.empty(5)
for train_idx, test_idx in kf.split(p_input, p_target):
    print(f"Started fold {fold}...")
    # Create the training sets from training indices
    X_cv_train, y_cv_train = p_input.iloc[train_idx], p_target.iloc[train_idx]
    # Create the test sets from test indices
    X_cv_test, y_cv_test = p_input.iloc[test_idx], p_target.iloc[test_idx]


NameError: name 'p_input' is not defined